In [73]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer # Or
from sklearn.linear_model import LogisticRegression # Or another
from sklearn.metrics import accuracy_score, classification_report
import nltk
from nltk.corpus import stopwords
import string
from textblob import TextBlob
from textblob import Word
# Download necessary NLTK resources (run this once)
nltk.download('stopwords')
nltk.download('punkt')


[nltk_data] Downloading package stopwords to /home/balong/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/balong/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [74]:
drugs = pd.read_csv('drugLibTrain_raw.tsv', sep = '\t') #pd.read_csv is used to load the dataframe however since the datta is in .tsv format, sep= '\t'
                                                        #is used because the values of a .tsv file is separated by tabs


In [75]:
drugs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3107 entries, 0 to 3106
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Unnamed: 0         3107 non-null   int64 
 1   urlDrugName        3107 non-null   object
 2   rating             3107 non-null   int64 
 3   effectiveness      3107 non-null   object
 4   sideEffects        3107 non-null   object
 5   condition          3106 non-null   object
 6   benefitsReview     3089 non-null   object
 7   sideEffectsReview  3032 non-null   object
 8   commentsReview     3095 non-null   object
dtypes: int64(2), object(7)
memory usage: 218.6+ KB


In [76]:
clean_drugs = drugs.dropna(subset=['sideEffectsReview'])

clean_drugs.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3032 entries, 0 to 3106
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Unnamed: 0         3032 non-null   int64 
 1   urlDrugName        3032 non-null   object
 2   rating             3032 non-null   int64 
 3   effectiveness      3032 non-null   object
 4   sideEffects        3032 non-null   object
 5   condition          3031 non-null   object
 6   benefitsReview     3017 non-null   object
 7   sideEffectsReview  3032 non-null   object
 8   commentsReview     3021 non-null   object
dtypes: int64(2), object(7)
memory usage: 236.9+ KB


In [77]:
def splitstring(x):
    x = " ".join(x.lower() for x in x.split()) 
    return x

clean_drugs['sideEffectsReview'] = clean_drugs['sideEffectsReview'].apply(splitstring)

clean_drugs.head(5)

/tmp/ipykernel_9410/3953916656.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_drugs['sideEffectsReview'] = clean_drugs['sideEffectsReview'].apply(splitstring)


,Unnamed: 0,urlDrugName,rating,effectiveness,sideEffects,condition,benefitsReview,sideEffectsReview,commentsReview
0,2202,enalapril,4,Highly Effective,Mild Side Effects,management of congestive heart failure,slowed the progression of left ventricular dys...,"cough, hypotension , proteinuria, impotence , ...","monitor blood pressure , weight and asses for ..."
1,3117,ortho-tri-cyclen,1,Highly Effective,Severe Side Effects,birth prevention,Although this type of birth control has more c...,"heavy cycle, cramps, hot flashes, fatigue, lon...","I Hate This Birth Control, I Would Not Suggest..."
2,1146,ponstel,10,Highly Effective,No Side Effects,menstrual cramps,I was used to having cramps so badly that they...,heavier bleeding and clotting than normal.,I took 2 pills at the onset of my menstrual cr...
3,3947,prilosec,3,Marginally Effective,Mild Side Effects,acid reflux,The acid reflux went away for a few months aft...,"constipation, dry mouth and some mild dizzines...",I was given Prilosec prescription at a dose of...
4,1951,lyrica,2,Marginally Effective,Severe Side Effects,fibromyalgia,I think that the Lyrica was starting to help w...,i felt extremely drugged and dopey. could not ...,See above


In [78]:
clean_drugs['sideEffectsReview'] = clean_drugs['sideEffectsReview'].str.replace('[^\w\s]', "", regex = True)

clean_drugs.head(5)

<>:1: SyntaxWarning: invalid escape sequence '\w'
<>:1: SyntaxWarning: invalid escape sequence '\w'
/tmp/ipykernel_9410/1225595197.py:1: SyntaxWarning: invalid escape sequence '\w'
  clean_drugs['sideEffectsReview'] = clean_drugs['sideEffectsReview'].str.replace('[^\w\s]', "", regex = True)
/tmp/ipykernel_9410/1225595197.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_drugs['sideEffectsReview'] = clean_drugs['sideEffectsReview'].str.replace('[^\w\s]', "", regex = True)


,Unnamed: 0,urlDrugName,rating,effectiveness,sideEffects,condition,benefitsReview,sideEffectsReview,commentsReview
0,2202,enalapril,4,Highly Effective,Mild Side Effects,management of congestive heart failure,slowed the progression of left ventricular dys...,cough hypotension proteinuria impotence rena...,"monitor blood pressure , weight and asses for ..."
1,3117,ortho-tri-cyclen,1,Highly Effective,Severe Side Effects,birth prevention,Although this type of birth control has more c...,heavy cycle cramps hot flashes fatigue long la...,"I Hate This Birth Control, I Would Not Suggest..."
2,1146,ponstel,10,Highly Effective,No Side Effects,menstrual cramps,I was used to having cramps so badly that they...,heavier bleeding and clotting than normal,I took 2 pills at the onset of my menstrual cr...
3,3947,prilosec,3,Marginally Effective,Mild Side Effects,acid reflux,The acid reflux went away for a few months aft...,constipation dry mouth and some mild dizziness...,I was given Prilosec prescription at a dose of...
4,1951,lyrica,2,Marginally Effective,Severe Side Effects,fibromyalgia,I think that the Lyrica was starting to help w...,i felt extremely drugged and dopey could not d...,See above


In [80]:
stop = stopwords.words('english')
def replace_stopwords(x):
    x = " ".join(x for x in x.split() if x not in stop)
    return x

    
clean_drugs['sideEffectsReview'].apply(replace_stopwords)

0       cough hypotension proteinuria impotence renal ...
1       heavy cycle cramps hot flashes fatigue long la...
2                        heavier bleeding clotting normal
3       constipation dry mouth mild dizziness would go...
4       felt extremely drugged dopey could drive med a...
                              ...                        
3102    restless legs night insomnia headache sometime...
3103    weight gain extreme tiredness day insomnia nig...
3104    constant issues patch staying called manufactu...
3105                             dizziness fatigue nausea
3106     find taking micardis tend tired libido decreased
Name: sideEffectsReview, Length: 3032, dtype: object